<a href="https://colab.research.google.com/github/AndoorAlanD/DA6401-Assignment-1/blob/main/Questions_3-8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wandb
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
os.environ['WANDB_SILENT']='true'

In [ ]:
wandb.login() #1ffc33d77af0fd022201ec32b81cd0e92cd75821

<IPython.core.display.Javascript object>

wandb: Paste an API key from your profile and hit enter:

 ··········


True

In [ ]:
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

X_train = X_train.reshape(X_train.shape[0], 28*28) / 255
X_test = X_test.reshape(X_test.shape[0], 28*28) /255
X_val = X_val.reshape(X_val.shape[0], 28*28) /255

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
class Feed_Forward_Neural_Network():

  def __init__(self, config):
    self.theta = []
    self.W = []
    self.b = []

    self.config = config
    self.eta = self.config['lr']

    self.l = 2 # input and output layers)
    self.L = []
    self.L.append(X_train.shape[1]) # input layer size

     # uniform hidden layers
    self.l += self.config['num_layers']
    for i in range(self.l-2):
      self.L.append(self.config['hidden_size'])
    self.L.append(np.max(y_train)+1) # output layer size

    if(config['weight_init']=='random'):
      for i in range(self.l-1):
        self.W.append(np.random.randn(self.L[i+1], self.L[i]))
        self.b.append(np.random.randn(self.L[i+1]))
    else:
      for i in range(self.l-1):
        xav_std = np.sqrt(2 / (self.L[i+1] + self.L[i]))
        self.W.append(np.random.randn(self.L[i+1], self.L[i]) * xav_std)
        self.b.append(np.random.randn(self.L[i+1]))

    if self.config['activation'] == 'sigmoid':
      self.act = self.sigmoid
      self.act_der = self.sigmoid_der
    elif self.config['activation'] == 'tanh':
      self.act = self.tanh
      self.act_der = self.tanh_der
    elif self.config['activation'] == 'ReLU':
      self.act = self.relu
      self.act_der = self.relu_der
    else:
      self.act = self.identity
      self.act_der = self.identity_der

    optimizers = {
        'stochastic': self.stochastic_gd,
        'momentum': self.momentum_gd,
        'nesterov': self.nest_acc_gd,
        'rmsprop': self.rmsprop,
        'adam': self.adam,
        'nadam': self.nadam
    }
    self.optimizer = optimizers[self.config['optimizer']](X_train, y_train)


  def identity(self, x):
    return x

  def identity_der(self, x):
    return np.ones(len(x))

  def sigmoid(self, x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

  def sigmoid_der(self, x):
    sg = self.sigmoid(x)
    return sg * (1 - sg)

  def tanh(self, x):
    return np.tanh(x)

  def tanh_der(self, x):
    new_x = self.tanh(x)
    return 1-(new_x*new_x)

  def relu(self, x):
    return np.maximum(0, x)

  def relu_der(self, x):
    return (x > 0).astype(int)

  def softmax(self, x):
    new_x = np.exp(np.clip(x,-500, 500))
    return  new_x / new_x.sum()


  def data_log(self, X=X_train, y=y_train):
    val_acc, val_true, val_pred = self.accuracy()
    train_acc, train_true, train_pred = self.accuracy(X, y)
    # val_acu=self.accuracy()
    # train_acu=self.accuracy(X,y)
    val_loss=self.cross_entropy_loss()
    train_loss=self.cross_entropy_loss(X,y)
    print(val_acc)
    wandb.log({
        'val_accuracy': val_acc,'val_loss': val_loss,'accuracy': train_acc,'loss': train_loss,
        'confusion_matrix': wandb.plot.confusion_matrix(probs=None,
                                                        y_true=val_true,
                                                        preds=val_pred,
                                                        class_names=[str(i) for i in range(np.max(y) + 1)])
    })
    # wandb.log({'val_accuracy': val_acu,'val_loss': val_loss,'accuracy': train_acu,'loss': train_loss})

  # def accuracy(self, X=X_val, y=y_val):
  #   correct_predictions = sum(np.argmax(self.feed_forward(x)[-1]) == y_true for x, y_true in zip(X, y))
  #   return (correct_predictions / len(y)) * 100.0

  def accuracy(self, X=X_val, y=y_val):
    y_pred_labels = []
    y_true_labels = []

    for x, y_true in zip(X, y):
        y_pred = np.argmax(self.feed_forward(x)[-1])
        y_pred_labels.append(y_pred)
        y_true_labels.append(y_true)

    correct_predictions = sum(y_p == y_t for y_p, y_t in zip(y_pred_labels, y_true_labels))
    acc = (correct_predictions / len(y)) * 100.0

    return acc, y_true_labels, y_pred_labels


  def squared_loss(self, X=X_val, y=y_val):
    sq_loss = 0
    for (x_in, y_true) in zip(X, y):
      _, _, y_pr = self.feed_forward(x_in)
      y_pr[y_true] -= 1
      sq_loss += np.sum(y_pr**2)
    return sq_loss / len(y)

  def cross_entropy_loss(self, X=X_val, y=y_val):
    loss = 0
    for (x_in, y_true) in zip(X, y):
      _, _, y_out = self.feed_forward(x_in)
      loss -= np.log(y_out[y_true]+1e-10)
    return loss / len(y)


  def feed_forward(self, x):
    pre_a = []
    act_h = [x]

    for i in range(self.l-2):
      pre_a.append(np.dot(self.W[i], act_h[i]) + self.b[i])
      act_h.append(self.act(pre_a[-1]))
    pre_a.append(self.b[-1] + np.dot(self.W[-1], act_h[-1]))
    y_pred = self.softmax(pre_a[-1])
    return pre_a, act_h, y_pred

  def back_prop(self, a, h, y, y_pred):
    gr_a = []

    if(self.config['loss'] == 'squared'):
      y_p = y_pred * (1-y_pred)
      y_pred[y] -= 1
      gr_a = np.array(y_p * y_pred)
    else:
      for i in range(len(y_pred)):
        if(i == y):
          gr_a.append(y_pred[i]-1)
        else:
          gr_a.append(y_pred[i])
      gr_a = np.array(gr_a)

    i = self.l-1
    gr_W, gr_b = [], []
    while i>0:
      gr_W.append(np.outer(gr_a, np.array(h[i-1])))
      gr_b.append(gr_a)
      if(i>1):
        gr_h = np.matmul(self.W[i-1].T, gr_a)
        gr_a = np.multiply(gr_h, self.act_der(a[i-2]))
      i -= 1

    return gr_W, gr_b

  def d_init(self):
    dw = [np.zeros_like(w) for w in self.W]
    db = [np.zeros_like(bi) for bi in self.b]
    return dw, db

  def stochastic_gd(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)

        for i in range(self.l-1):
            self.W[i] = np.array(self.W[i]) - self.eta * np.array(grad_W[-1-i])
            self.b[i] = np.array(self.b[i]) - self.eta * np.array(grad_b[-1-i])

      self.data_log()

  def momentum_gd(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']
    batch_size = self.config['batch_size']
    decay = self.config['weight_decay']
    beta = 0.99
    w_old, b_old = self.d_init()
    index = 0

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")
      dw, db = self.d_init()

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)

        for i in range(self.l-1):
          dw[i] += grad_W[-1-i]
          db[i] += grad_b[-1-i]

        index += 1
        if(index == batch_size):
          for i in range(self.l-1):
            dw[i] += decay*self.W[i]
            w_old[i] = beta*w_old[i] + self.eta*dw[i]
            b_old[i] = beta*b_old[i] + self.eta*db[i]
            self.W[i] -= w_old[i]
            self.b[i] -= b_old[i]
          dw, db = self.d_init()
          index=0

      self.data_log()

  def nest_acc_gd(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']
    batch_size = self.config['batch_size']
    decay = self.config['weight_decay']
    beta = 0.9
    vw_old, vb_old = self.d_init()
    index = 0

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")
      dw, db = self.d_init()

      for i in range(self.l-1):
        self.W[i] -= beta*vw_old[i]
        self.b[i] -= beta*vb_old[i]

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)

        for i in range(self.l-1):
          dw[i] += grad_W[-1-i]
          db[i] += grad_b[-1-i]


        for i in range(self.l-1):
          vw_old[i] = beta*vw_old[i] + self.eta*dw[i]
          vb_old[i] = beta*vb_old[i] + self.eta*db[i]

        index += 1
        if(index == batch_size):
          for i in range(self.l-1):
            dw[i] += decay*self.W[i]
            self.W[i] -= dw[i]
            self.b[i] -= db[i]
          dw, db = self.d_init()
          index=0

      self.data_log()


  def rmsprop(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']
    batch_size = self.config['batch_size']
    decay = self.config['weight_decay']
    beta = 0.9
    eps = 1e-8
    vw, vb = self.d_init()
    index = 0

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")
      dw, db = self.d_init()

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)
        for i in range(self.l-1):
          dw[i] += grad_W[-1-i]
          db[i] += grad_b[-1-i]

        index += 1
        if(index == batch_size):
          for i in range(self.l-1):
            dw[i] += decay*self.W[i]
            vw[i] = beta*vw[i] + (1-beta)*(dw[i]**2)
            vb[i] = beta*vb[i] + (1-beta)*(db[i]**2)
            self.W[i] -= self.eta*dw[i] / (np.sqrt(vw[i])+eps)
            self.b[i] -= self.eta*db[i] / (np.sqrt(vb[i])+eps)
          dw, db = self.d_init()
          index=0

      self.data_log()

  def adam(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']
    batch_size = self.config['batch_size']
    decay = self.config['weight_decay']
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-5
    index = 0

    mw, mb = self.d_init()
    vw, vb = self.d_init()
    mw_hat, mb_hat = self.d_init()
    vw_hat, vb_hat = self.d_init()

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")
      dw, db = self.d_init()

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)
        for i in range(self.l-1):
            dw[i] += grad_W[-1-i]
            db[i] += grad_b[-1-i]

        index += 1
        if(index == batch_size):
          for i in range(self.l-1):
            dw[i] += decay*self.W[i]
            mw[i] = beta1*mw[i] + (1-beta1)*dw[i]
            mb[i] = beta1*mb[i] + (1-beta1)*db[i]
            vw[i] = beta2*vw[i] + (1-beta2)*dw[i]**2
            vb[i] = beta2*vb[i] + (1-beta2)*db[i]**2
            mw_hat[i] = mw[i] / (1-np.power(beta1, ep+1))
            mb_hat[i] = mb[i] / (1-np.power(beta1, ep+1))
            vw_hat[i] = vw[i] / (1-np.power(beta2, ep+1))
            vb_hat[i] = vb[i] / (1-np.power(beta2, ep+1))
            self.W[i] -= self.eta*mw_hat[i] / (np.sqrt(vw_hat[i])+eps)
            self.b[i] -= self.eta*mb_hat[i] / (np.sqrt(vb_hat[i])+eps)
          dw, db = self.d_init()
          index=0

      self.data_log()

  def nadam(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']
    batch_size = self.config['batch_size']
    decay = self.config['weight_decay']
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-5
    mw, mb = self.d_init()
    vw, vb = self.d_init()
    mw_hat, mb_hat = self.d_init()
    vw_hat, vb_hat = self.d_init()
    index = 0

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")
      dw, db = self.d_init()

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)
        for i in range(self.l-1):
            dw[i] += grad_W[-1-i]
            db[i] += grad_b[-1-i]

        index += 1
        if(index == batch_size):
          for i in range(self.l-1):
            dw[i] += decay*self.W[i]
            mw[i] = beta1*mw[i] + (1-beta1)*dw[i]
            mb[i] = beta1*mb[i] + (1-beta1)*db[i]
            vw[i] = beta2*vw[i] + (1-beta2)*dw[i]**2
            vb[i] = beta2*vb[i] + (1-beta2)*db[i]**2
            mw_hat[i] = mw[i] / (1-np.power(beta1, ep+1))
            mb_hat[i] = mb[i] / (1-np.power(beta1, ep+1))
            vw_hat[i] = vw[i] / (1-np.power(beta2, ep+1))
            vb_hat[i] = vb[i] / (1-np.power(beta2, ep+1))
            self.W[i] -= (self.eta/(np.sqrt(vw_hat[i])+eps)) * (beta1*mw_hat[i] + (1-beta1)*dw[i]/(1-beta1**(ep+1)))
            self.b[i] -= (self.eta/(np.sqrt(vb_hat[i])+eps)) * (beta1*mb_hat[i] + (1-beta1)*db[i]/(1-beta1**(ep+1)))
          dw, db = self.d_init()
          index=0

      self.data_log()


In [6]:
import wandb
import random

sweep_config = {
    'method': 'bayes',
    'name' : 'Trial_1',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },

        'num_layers': {
            'values': [3,4,5]
        },
        'hidden_size': {
            'values': [32, 64, 128]
        },

        'weight_decay': {
            'values': [0, 0.0005, 0.05]
        },

        'lr': {
            'values': [0.001, 0.0001]
        },

        'optimizer': {
            'values': ["stochastic", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
        },

        'batch_size': {
            'values': [16,32,64]
        },

        'weight_init': {
            'values': ['random', 'xavier']
        },
        'activation': {
            'values': ["identity", "sigmoid", "tanh", "ReLU"]
        },

        'loss': {
            'values': ['cross_entropy', 'squared']
        },

    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A1')

Create sweep with ID: r5b912dw
Sweep URL: https://wandb.ai/alandandoor-iit-madras/DL_A1/sweeps/r5b912dw


In [ ]:
def main():

    with wandb.init() as run:
        wandb.run.name = (f"{wandb.config.epochs}_{wandb.config.optimizer}_{wandb.config.activation}_{wandb.config.loss}_{wandb.config.batch_size}_{wandb.config.num_layers}_{wandb.config.lr}_{wandb.config.weight_init}")
        model = Feed_Forward_Neural_Network(wandb.config)
        print(model.accuracy(X_test, y_test))

wandb.agent(sweep_id, function=main, count=5)
#, count=50
# wandb.finish()

Epoch 1/10
81.15
Epoch 2/10
82.43333333333334
Epoch 3/10
82.88333333333333
Epoch 4/10
82.91666666666667
Epoch 5/10
82.81666666666668
Epoch 6/10
82.93333333333334
Epoch 7/10
83.1
Epoch 8/10
83.3
Epoch 9/10
83.41666666666666
Epoch 10/10
83.55
(82.23, [9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9, 1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1, 2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0, 2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8, 0, 1, 4, 2, 3, 6, 7, 2, 7, 8, 5, 9, 9, 4, 2, 5, 7, 0, 5, 2, 8, 6, 7, 8, 0, 0, 9, 9, 3, 0, 8, 4, 1, 5, 4, 1, 9, 1, 8, 6, 2, 1, 2, 5, 1, 0, 0, 0, 1, 6, 1, 6, 2, 2, 4, 4, 1, 4, 5, 0, 4, 7, 9, 3, 7, 2, 3, 9, 0, 9, 4, 7, 4, 2, 0, 5, 2, 1, 2, 1, 3, 0, 9, 1, 0, 9, 3, 6, 7, 9, 9, 4, 4, 7, 1, 2, 1, 6, 3, 2, 8, 3, 6, 1, 1, 0, 2, 9, 2, 4, 0, 7, 9, 8, 4, 1, 8, 4, 1, 3, 1, 6, 7, 2, 8, 5, 2, 0, 7, 7, 6, 2, 7, 0, 7, 8, 9, 2, 9, 0, 5, 1, 4, 4, 5, 6, 9, 2, 6, 8, 6, 4, 2, 2, 

Epoch 1/5
61.68333333333334
Epoch 2/5
65.93333333333334
Epoch 3/5
66.85
Epoch 4/5
68.35
Epoch 5/5
69.88333333333333
(69.39, [9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9, 1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1, 2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0, 2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8, 0, 1, 4, 2, 3, 6, 7, 2, 7, 8, 5, 9, 9, 4, 2, 5, 7, 0, 5, 2, 8, 6, 7, 8, 0, 0, 9, 9, 3, 0, 8, 4, 1, 5, 4, 1, 9, 1, 8, 6, 2, 1, 2, 5, 1, 0, 0, 0, 1, 6, 1, 6, 2, 2, 4, 4, 1, 4, 5, 0, 4, 7, 9, 3, 7, 2, 3, 9, 0, 9, 4, 7, 4, 2, 0, 5, 2, 1, 2, 1, 3, 0, 9, 1, 0, 9, 3, 6, 7, 9, 9, 4, 4, 7, 1, 2, 1, 6, 3, 2, 8, 3, 6, 1, 1, 0, 2, 9, 2, 4, 0, 7, 9, 8, 4, 1, 8, 4, 1, 3, 1, 6, 7, 2, 8, 5, 2, 0, 7, 7, 6, 2, 7, 0, 7, 8, 9, 2, 9, 0, 5, 1, 4, 4, 5, 6, 9, 2, 6, 8, 6, 4, 2, 2, 9, 7, 6, 5, 5, 2, 8, 5, 2, 3, 0, 4, 8, 0, 0, 6, 3, 8, 9, 6, 1, 3, 0, 2, 3, 0, 8, 3, 7, 4, 0, 1, 6, 3, 0, 6, 6, 7, 5, 4, 5, 9,

Epoch 1/10
10.05
Epoch 2/10
10.316666666666666
Epoch 3/10
10.616666666666667
Epoch 4/10
10.299999999999999
Epoch 5/10
10.05
Epoch 6/10
10.05
Epoch 7/10
10.299999999999999
Epoch 8/10
10.299999999999999
Epoch 9/10
